# parallel-chirts-time-series-zonal-statistics

Like [parallel-chirps-time-series-zonal-statistics](https://github.com/alex-pakalniskis/parallel-chirps-time-series-zonal-statistics), but with [Climate Hazards Center Infrared Temperature with Stations (CHIRTSmax)](https://www.chc.ucsb.edu/data/chirts) data (30+ GB)

Requires that raster file names contain datetime index information in between the `file_prefix` and `file_type_of_interest`, i.e. `file_prefix.2019.11.file_type_of_interest` or `file_prefix.1960.02.file_type_of_interest`

### Inputs and Output

In [1]:
boundaries = "/home/alex/data/chirps_africa_monthly/not_monthly_tif/zambia_districts_cleaned.geojson"

raster_folder = "/home/alex/data/chirts"

statistic_of_interest = "mean"

identifier_column = "Dist_name"

file_prefix = "CHIRTSmax."

file_type_of_interest = ".tif"

output_csv = "/home/alex/data/chirts/zambia-districts-chirts-zonal-statistics.csv"


### Libraries

In [2]:
import os

import glob 

import numpy as np

import rasterio 

from rasterstats import zonal_stats

import pandas as pd

import geopandas as gpd

import multiprocessing

from datetime import datetime

### Processing

In [3]:
results = {}

os.chdir(raster_folder)

def processing(name): 
    
    result = zonal_stats(boundaries, name, stats=[statistic_of_interest], nodata=-9999, all_touched=True, geojson_out=False) 
    
    return (name, result)  

start = datetime.now()

with multiprocessing.Pool(multiprocessing.cpu_count()) as pool: 
    
    for name, result in pool.map(processing, glob.glob("*" + file_type_of_interest)): 
        
        results[name] = result
    
value_list = []

for i in range(len(list(results.values()))):
    
    for j in range(len(list(results.values())[i])):
        
        value_list.append(list(list(results.values())[i][j].values()))
        
key_list = []

for i in range(len(list(results.values()))):
    
    for j in range(len(list(results.values())[i])):
        
        key_list.append(list(results.keys())[i])
        
cleaned_outputs = pd.DataFrame(np.array(value_list), index=key_list, columns = [statistic_of_interest])

cleaned_outputs = cleaned_outputs.reset_index()

cleaned_outputs["Date"] = pd.to_datetime(cleaned_outputs["index"].str.split(file_prefix).str[1].str.split(file_type_of_interest).str[0].str.replace(".","-"))

cleaned_outputs = cleaned_outputs.drop("index", axis=1)

cleaned_outputs[identifier_column] = pd.Series(np.tile(gpd.read_file(boundaries)[identifier_column].unique().tolist(), len([name for name in os.listdir(raster_folder) if os.path.isfile(os.path.join(raster_folder, name))])))

cleaned_outputs = cleaned_outputs.sort_values(by="Date").set_index("Date")

cleaned_outputs.to_csv(output_csv)

stop = datetime.now()

print("Execution time for\n\nMulticore, multi-band, multi-polygon zonal statistics\n\n{}".format(str(stop - start)))


Execution time for

Multicore, multi-band, multi-polygon zonal statistics

0:01:41.795191
